<a href="https://colab.research.google.com/github/vinileodido/MVP_PucRio_ML/blob/main/ML_IoT_Industrial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Manutenção Preditiva com Machine Learning
## Dataset IoT Industrial

## 1. Instalação de Pacotes Necessários

In [ ]:
# Instalar pacotes necessários (executar apenas se necessário)
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Lista de pacotes necessários
required_packages = ['xgboost', 'imbalanced-learn', 'plotly']

for package in required_packages:
    try:
        __import__(package.replace('-', '_'))
        print(f"✓ {package} já instalado")
    except ImportError:
        print(f"Instalando {package}...")
        install_package(package)
        print(f"✓ {package} instalado com sucesso")

## 2. Importação das Bibliotecas

In [ ]:
# Bibliotecas básicas
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Plotly (com tratamento de erro)
try:
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    PLOTLY_AVAILABLE = True
except ImportError:
    print("⚠️ Plotly não disponível. Usando apenas matplotlib/seaborn")
    PLOTLY_AVAILABLE = False

# Pré-processamento
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.impute import SimpleImputer

# Modelos
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsRegressor

# XGBoost (com tratamento de erro)
try:
    from xgboost import XGBClassifier, XGBRegressor
    XGBOOST_AVAILABLE = True
    print("✓ XGBoost disponível")
except ImportError:
    print("⚠️ XGBoost não disponível. Use: pip install xgboost")
    XGBOOST_AVAILABLE = False
    class XGBClassifier:
        def __init__(self, **kwargs):
            raise NotImplementedError("XGBoost não está instalado. Use: pip install xgboost")
    class XGBRegressor:
        def __init__(self, **kwargs):
            raise NotImplementedError("XGBoost não está instalado. Use: pip install xgboost")

# LightGBM (com tratamento de erro)
try:
    import lightgbm as lgb
    from lightgbm import LGBMRegressor
    LIGHTGBM_AVAILABLE = True
    print("✓ LightGBM disponível")
except ImportError:
    print("⚠️ LightGBM não disponível. Use: pip install lightgbm")
    LIGHTGBM_AVAILABLE = False
    class LGBMRegressor:
        def __init__(self, **kwargs):
            raise NotImplementedError("LightGBM não está instalado. Use: pip install lightgbm")

# Métricas
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score,
                            roc_curve, precision_recall_curve, f1_score, accuracy_score,
                            mean_absolute_error, mean_squared_error, r2_score)

# Balanceamento (com tratamento de erro)
try:
    from imblearn.over_sampling import SMOTE
    from imblearn.under_sampling import RandomUnderSampler
    from imblearn.pipeline import Pipeline as ImbPipeline
    IMBLEARN_AVAILABLE = True
    print("✓ Imbalanced-learn disponível")
except ImportError:
    print("⚠️ Imbalanced-learn não disponível. Use: pip install imbalanced-learn")
    IMBLEARN_AVAILABLE = False

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("\n" + "="*60)
print("BIBLIOTECAS CARREGADAS COM SUCESSO!")
print("="*60)